In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
#from statsmodels.tsa.vector_ar.var_model import VARY
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed

import pyarrow
import concurrent.futures

In [ ]:
import myccm
importlib.reload(myccm)

In [ ]:
#sites = ['MCRI', 'IUR', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'KUMC', 'UTSW']
sites = ['IUR', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'KUMC', 'UTSW']

In [ ]:
loincs_cofounder_potassium = ['2157-6', #creatine kinase
                              '1920-8', #AST      
                              '2532-0', #LDH
                              '4542-7', #Haptoglobin 
                              '3084-1'] #uric acid
loincs_potassium = ['2823-3', '2160-0']

In [ ]:
import myccm
labXs = list()
for site in sites:
    try:
        print(site)
        myccmX = myccm.myccm()
        myccmX.get_onsetdata(site=site)                
        myccmX.get_data(site=site)      
        labt = myccmX.labX
        labt['site'] = site
        labXs.append(labt)
    except:
       print('not working')
    # for loinc1 in loincs_cofounder_potassium:
    #     for loinc2 in loincs_potassium:
    #         try:
    #             generate_data_for_pco(loinc1, loinc2)
    #         except:
    #             pass

In [ ]:
def process_site(site):
    try:
        print(site)
        myccmX = myccm.myccm()
        myccmX.get_onsetdata(site=site)                
        myccmX.get_data(site=site)      
        labt = myccmX.labX
        labt['site'] = site
        return labt    
    except Exception as e: 
        print('not working for site:', site, 'Error:', e)

def parallel_processing(sites):
    labXs = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = executor.map(process_site, sites)        
        for result in results:
            labXs.extend(result)
    return labXs

labXs = parallel_processing(sites)

In [ ]:
def generate_data_for_pco(loinc1, loinc2, labX):
    outputlab = labX[labX['LAB_LOINC'].str.contains('|'.join([loinc1,loinc2])).fillna(False)]

    outputlab = outputlab[['ENCOUNTERID','DAYS_SINCE_ADMIT','RESULT_NUM','LAB_LOINC']]

    outputlab = outputlab.groupby(['ENCOUNTERID','DAYS_SINCE_ADMIT','LAB_LOINC']).mean().reset_index()

    xxx = outputlab.groupby(['ENCOUNTERID','LAB_LOINC'])['DAYS_SINCE_ADMIT'].count().reset_index().pivot(index='ENCOUNTERID', columns='LAB_LOINC', values='DAYS_SINCE_ADMIT').dropna()

    xxx = xxx[(xxx[loinc1]>=6)&(xxx[loinc2]>=6)]

    outputlab2 = xxx.merge(outputlab, on='ENCOUNTERID', how='left')

    xxx2 = outputlab2.groupby(['ENCOUNTERID','LAB_LOINC'])['DAYS_SINCE_ADMIT'].max().reset_index().pivot(index='ENCOUNTERID', columns='LAB_LOINC', values='DAYS_SINCE_ADMIT').dropna()

    pltdf = xxx2.groupby(loinc1).count()

    outputlab21 = outputlab2[outputlab2['LAB_LOINC']==loinc1]
    outputlab22 = outputlab2[outputlab2['LAB_LOINC']==loinc2]

    outputlab21 = outputlab21[['ENCOUNTERID','DAYS_SINCE_ADMIT','RESULT_NUM']]
    outputlab21 = outputlab21.pivot(index='DAYS_SINCE_ADMIT', columns='ENCOUNTERID', values='RESULT_NUM')
    outputlab21 = outputlab21.interpolate(limit_direction='both')
    outputlab21 = outputlab21[outputlab21.index<=7]

    outputlab22 = outputlab22[['ENCOUNTERID','DAYS_SINCE_ADMIT','RESULT_NUM']]
    outputlab22 = outputlab22.pivot(index='DAYS_SINCE_ADMIT', columns='ENCOUNTERID', values='RESULT_NUM')
    outputlab22 = outputlab22.interpolate(limit_direction='both')
    outputlab22 = outputlab22[outputlab22.index<=7]

    datafolder = '/home/hoyinchan/blue/Data/data2021raw/'

    outputlab21.columns = outputlab21.columns.astype(str)
    outputlab22.columns = outputlab22.columns.astype(str)

    outputlab21.to_parquet(datafolder+site+'outputlab'+loinc1+loinc2+'.parquet')
    outputlab22.to_parquet(datafolder+site+'outputlab'+loinc2+loinc1+'.parquet')